In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, Input,Attention, concatenate,Dropout,GRU

In [4]:
poem = []
with open('/content/drive/MyDrive/ferdousi.txt') as f:
    poem = f.readlines()

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(poem)

In [6]:

# convert the tokenized data to sequences
sequences = tokenizer.texts_to_sequences(poem)

In [7]:
# split the data into odd hemistich (encoder input) and even hemistich (decoder input)
encoder_input_data = [sequences[i] for i in range(0, len(sequences), 2)]
decoder_input_data = [sequences[i] for i in range(1, len(sequences), 2)]


In [8]:

# pad the encoder and decoder input data
max_len = max(len(s) for s in encoder_input_data + decoder_input_data)
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_len, padding='pre')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_len, padding='post')
decoder_input_data = decoder_input_data[:,:-1]
encoder_input_data = encoder_input_data[:49608,:]
print(decoder_input_data.shape)
encoder_input_data.shape

(49608, 10)


(49608, 11)

In [9]:
encoder_input_data = np.concatenate((encoder_input_data, np.zeros((encoder_input_data.shape[0], 1))), axis=1)

In [11]:

# encoder model
encoder_inputs = Input(shape=(12,))
encoder_embedding = Embedding(len(tokenizer.word_index) + 1, 10)(encoder_inputs)
encoder_embedding = Dropout(0.2)(encoder_embedding)
encoder_lstm = LSTM(10, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]
encoder_states = Dropout(0.2)(encoder_states) # add dropout here

In [12]:

# decoder model
decoder_inputs = Input(shape=(10,))
decoder_embedding = Embedding(len(tokenizer.word_index) + 1, 10)(decoder_inputs)
decoder_embedding = Dropout(0.2)(decoder_embedding)
decoder_lstm = LSTM(10, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[encoder_states[0], encoder_states[1]])
decoder_outputs = Dropout(0.2)(decoder_outputs) # add dropout here
#decoder_dense = Dense(len(tokenizer.word_index) + 1, activation='softmax')
#decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
# Attention Layer
attention = Attention(name='attention')([decoder_outputs, encoder_outputs])
decoder_concat = concatenate([decoder_outputs, attention], axis=-1)

In [14]:
# Dense layer
decoder_dense = Dense(len(tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat)

In [15]:

# compile the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr=0.1)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
# fit the model on the datafrom 
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='min')

model.fit([encoder_input_data, decoder_input_data], decoder_input_data, epochs=4, batch_size=64, verbose=2, validation_split=0.3, callbacks=[early_stopping])


Epoch 1/4
543/543 - 23s - loss: 3.2661 - accuracy: 0.5228 - val_loss: 2.5707 - val_accuracy: 0.6260 - 23s/epoch - 42ms/step
Epoch 2/4
543/543 - 15s - loss: 2.7814 - accuracy: 0.5582 - val_loss: 2.4587 - val_accuracy: 0.6356 - 15s/epoch - 27ms/step
Epoch 3/4
543/543 - 13s - loss: 2.7088 - accuracy: 0.5621 - val_loss: 2.3812 - val_accuracy: 0.6433 - 13s/epoch - 25ms/step
Epoch 4/4
543/543 - 14s - loss: 2.6589 - accuracy: 0.5666 - val_loss: 2.3465 - val_accuracy: 0.6458 - 14s/epoch - 25ms/step


**GRU**

In [18]:
# encoder model
encoder_inputs = Input(shape=(12,))
encoder_embedding = Embedding(len(tokenizer.word_index) + 1, 10)(encoder_inputs)
encoder_embedding = Dropout(0.2)(encoder_embedding)
encoder_gru = GRU(10, return_state=True)
encoder_outputs, state_h = encoder_gru(encoder_embedding)
encoder_states = [state_h]
encoder_states = Dropout(0.2)(encoder_states)



In [19]:

# decoder model
decoder_inputs = Input(shape=(10,))
decoder_embedding = Embedding(len(tokenizer.word_index) + 1, 10)(decoder_inputs)
decoder_embedding = Dropout(0.2)(decoder_embedding)
decoder_gru = GRU(10, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=[encoder_states[0]])
decoder_outputs = Dropout(0.2)(decoder_outputs)

In [20]:

# Attention Layer
attention = Attention(name='attention')([decoder_outputs, encoder_outputs])
decoder_concat = concatenate([decoder_outputs, attention], axis=-1)

In [21]:
# Dense layer
decoder_dense = Dense(len(tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat)

In [22]:

# compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [23]:

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='min')

model.fit([encoder_input_data, decoder_input_data], decoder_input_data, epochs=4, batch_size=64, verbose=2, validation_split=0.3, callbacks=[early_stopping])

Epoch 1/4
543/543 - 18s - loss: 2.6228 - accuracy: 0.5683 - val_loss: 2.3335 - val_accuracy: 0.6519 - 18s/epoch - 34ms/step
Epoch 2/4
543/543 - 14s - loss: 2.6031 - accuracy: 0.5708 - val_loss: 2.3229 - val_accuracy: 0.6550 - 14s/epoch - 26ms/step
Epoch 3/4
543/543 - 14s - loss: 2.5917 - accuracy: 0.5697 - val_loss: 2.3215 - val_accuracy: 0.6552 - 14s/epoch - 26ms/step
Epoch 4/4
543/543 - 15s - loss: 2.6004 - accuracy: 0.5710 - val_loss: 2.3455 - val_accuracy: 0.6489 - 15s/epoch - 27ms/step
